In [37]:
import requests
import json
import csv
import pandas as pd
import urllib.parse
import numpy as np

pd.set_option('display.max_columns', None)       # Show all columns
pd.set_option('display.width', None)

In [38]:
in_file = '2023_fires_in_tempe_loc1.csv'
out_file = '2023_fires_in_tempe.json'

In [39]:
with open('secrets/gmaps_key', 'r') as f:
    api_key = f.read()

In [40]:
with open(in_file,'r') as f:
    df = pd.read_csv(f)

# df = df.iloc[:,1:]
# df['_addr'] = df['_addr'].map(lambda val: urllib.parse.quote(val))
df

responses = pd.read_pickle('api_response.pkl')

In [41]:
responses = []
url = 'https://maps.googleapis.com/maps/api/geocode/json'

for i,row in df.iterrows():
    params = {
        'key': api_key,
        'outputFormat': 'json',
        'address': row['_addr']
    }
    print(row['_addr'])
    response = requests.get(url, params=params)
    if response.status_code == 200:
        responses.append(response.json())
        print(response.json())
    else:
        print(f"Error {response.status_code}")
    # break

pd.to_pickle(responses, 'api_response.pkl')

with open('api_response.json','w') as f:
    f.write(json.dumps(responses))

9631 S LA ROSA DR Tempe AZ 85284
{'results': [{'address_components': [{'long_name': '9631', 'short_name': '9631', 'types': ['street_number']}, {'long_name': 'South La Rosa Drive', 'short_name': 'S La Rosa Dr', 'types': ['route']}, {'long_name': 'South Tempe', 'short_name': 'South Tempe', 'types': ['neighborhood', 'political']}, {'long_name': 'Tempe', 'short_name': 'Tempe', 'types': ['locality', 'political']}, {'long_name': 'Maricopa County', 'short_name': 'Maricopa County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Arizona', 'short_name': 'AZ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '85284', 'short_name': '85284', 'types': ['postal_code']}, {'long_name': '4022', 'short_name': '4022', 'types': ['postal_code_suffix']}], 'formatted_address': '9631 S La Rosa Dr, Tempe, AZ 85284, USA', 'geometry': {'bounds': {'northeast': {'lat': 33.3246392, 'lng

In [42]:
lat = []
lng = []
for r in responses:
    lat.append(r['results'][0]['geometry']['location']['lat'])
    lng.append(r['results'][0]['geometry']['location']['lng'])

In [43]:
df['lat'] = lat
df['lng'] = lng
df

,INCIDENT_KEY,_addr,_desc,INC_DATE,lat,lng
0,AZ_08052_01102023_231089_0,9631 S LA ROSA DR Tempe AZ 85284,Number of Residential Units: 1.0\nArea of Orig...,2023-01-10,33.324533,-111.930831
1,AZ_08293_01102023_23841_0,6230 S TAYLOR DR Tempe AZ 85283,Number of Residential Units: 1.0\nArea of Orig...,2023-01-10,33.364401,-111.908903
2,AZ_08293_01202023_231742_0,1512 N SCOTTSDALE RD Tempe AZ 85281,"Area of Origin: Engine area, running gear, whe...",2023-01-20,33.446199,-111.926981
3,AZ_08293_02012023_232753_0,485 W CARMEN ST Tempe AZ 85283,Number of Residential Units: 1.0\nNumber of Bu...,2023-02-01,33.359411,-111.948662
4,AZ_08293_02072023_233243_0,1245 E LAGUNA DR Tempe AZ 85282,Number of Residential Units: 1.0\nArea of Orig...,2023-02-07,33.390257,-111.919445
...,...,...,...,...,...,...
249,AZ_08293_12252023_2333577_0,636 W MANHATTON DR Tempe AZ 85282,Area of Origin: Undetermined\nHeat Source: Und...,2023-12-25,33.389506,-111.949217
250,AZ_08293_12262023_2333657_0,40 E GAMMAGE PKY Tempe AZ 85281,Not Residential Flag: Y\nNumber of Buildings I...,2023-12-26,33.418065,-111.939389
251,AZ_08293_12272023_2333718_0,3141 S MCCLINTOCK DR Tempe AZ 85282,Number of Residential Units: 0.0\nNumber of Bu...,2023-12-27,33.394784,-111.908588
252,AZ_08293_12302023_2333948_0,2425 E APACHE BLVD Tempe AZ 85281,Number of Residential Units: 1.0\nNumber of Bu...,2023-12-30,33.414660,-111.886196


In [44]:
import geojson

features = []

for i, row in df.iterrows():
    p = geojson.Point((row['lat'], row['lng']))
    f = geojson.Feature(id=i, geometry=p, properties={
        "description": row['_desc'],
        "date": row['INC_DATE'],
        "address": row['_addr']
    })
    features.append(f)

fc = geojson.FeatureCollection(features=features)

with open(out_file,'w') as f:
    f.write(geojson.dumps(fc))

In [45]:
# responses = []
# url = 'https://maps.googleapis.com/maps/api/elevation/json'

# # for i, row in df.iterrows():
# #     params = {
# #         'key': api_key,
# #         'locations': f"{str(row['lat'])}, {str(row['lng'])}"
# #     }
# #     response = requests.get(url, params=params)
# #     if response.status_code == 200:
# #         responses.append(response.json())
# #         print(response.json())
# #     else:
# #         print(f"Error {response.status_code}")

In [46]:
# elevations = []
# for row in responses:
#     elevations.append(row['results'][0]['elevation'])

In [47]:
# df['elevation'] = elevations

In [48]:
# df.to_json('nfirs_cleaned_forunity.json',orient='records')